In [114]:
import pandas as pd
import pandas_ta as ta
import random
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Flatten, Activation
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
import statistics
from sklearn import preprocessing
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os.path
import yfinance as yf

In [13]:
ticker = "TNA"

In [129]:
def getDB(ticker):
    tick = ticker
    # Load data
    data = yf.Ticker(tick)
    df = data.history(period="1y", interval="1d")
#     df = data.history(start="2018-12-01", end="2020-03-01")
#     start="2017-01-01", end="2017-04-30"
    
    # add data points
    df['close_per1'] = df.ta.percent_return(1)*100
    df['sma10'] = df.ta.sma(length=10)
    df['williams'] = df.ta.willr(7)


    df = df[[
            'open','close','sma10','williams','close_per1'
            ]]

    df = df.dropna()
    df.reset_index(inplace=True)
    
    return df

In [130]:
ticker = "NAIL"
df = getDB(ticker)
print(df.head(10))
df.iloc[0].index

        date       open      close      sma10   williams  close_per1
0 2021-01-15  47.340000  48.799999  46.416000 -32.407401    1.624321
1 2021-01-19  49.820000  51.619999  47.194999  -3.670765    5.778688
2 2021-01-20  52.000000  58.189999  48.678999  -8.429536   12.727625
3 2021-01-21  59.599998  60.689999  50.299999 -11.297088    4.296271
4 2021-01-22  59.070000  62.419998  51.702999  -1.745652    2.850551
5 2021-01-25  62.150002  63.279999  53.569999 -14.594057    1.377765
6 2021-01-26  64.370003  60.189999  54.903999 -33.643960   -4.883060
7 2021-01-27  57.259998  56.040001  55.700999 -63.239516   -6.894829
8 2021-01-28  56.630001  56.380001  56.562999 -70.798221    0.606710
9 2021-01-29  55.220001  53.570000  57.117999 -86.244396   -4.984039


Index(['date', 'open', 'close', 'sma10', 'williams', 'close_per1'], dtype='object')

In [145]:
# tickers = ["FAS","TNA","SOXL","TECL","LABU","YINN","DPST","NAIL"]
# tickers = ["EDC"]
# tickers = ["SOXL","SPXL","TECL","FAS","UPRO","FNGU","TNA","LABU","YINN","DPST","NAIL","DFEN","CURE","EURL","DUSL","MIDU","EDU","MEXX","PILL","DRN","RETL","TPOR","UTSL","KORU","OILU"]
tickers = ["SOXL","FAS","FNGU","TNA","LABU","YINN","DPST","NAIL","DFEN","DUSL","MIDU","MEXX","PILL","RETL","TPOR","KORU"]

ALL = []; amount = []
for ticker in tickers:
    print()
    print("ticker: {}".format(ticker))
    df = getDB(ticker)
    ## Williams trading idea ##
    returnP1 = []; returnP2 = []
    buyState = False
    # setting the outputs in the df
    for i in range(len(df)):
        if df.iloc[i]['williams'] < -80:
            buy = df.iloc[i]['close']
            buyState = True
            buyD = df.iloc[i]['date']

        if buyState == True and df.iloc[i]['williams'] > -20:
            sell = df.iloc[i]['close']
            R = ((sell-buy)/abs(buy))*100
            r = round(R, 2)
            returnP1.append(r)
            buyState = False
            sellD = df.iloc[i]['date']
            delta = (sellD - buyD).days
            print("Bought on {} & sold on {}. Delta {} days. {}%".format(buyD.strftime('%Y-%m-%d'), sellD.strftime('%Y-%m-%d'), delta, r))

#     print()
#     print()
    avgP1 = np.average(returnP1)
    # avgP2 = np.average(returnP2)
    # print(avgP1)
#     print("ticker: {}".format(ticker))
#     print("Number of trade days: %d" % (len(df)))
    print("Number of trades: %d" % (len(returnP1)))
    print("Percentage average when price opens higher than previous close: %f" % (avgP1))
    # print("Average return if loss limit is 0.1 percent: %f" % (avgP2))
#     print(returnP1)
    ALL.append(avgP1)
    amount.append(len(returnP1))
    time.sleep(3)
    
print("total average: {}".format(np.average(ALL)))
print("total trades: {}".format(int(np.sum(amount))))
print("with {} tickers".format(len(tickers)))


ticker: SOXL
Bought on 2021-01-29 & sold on 2021-02-08. Delta 10 days. 20.16%
Bought on 2021-03-08 & sold on 2021-03-15. Delta 7 days. 29.25%
Bought on 2021-03-24 & sold on 2021-03-26. Delta 2 days. 15.65%
Bought on 2021-05-13 & sold on 2021-05-19. Delta 6 days. 9.27%
Bought on 2021-06-18 & sold on 2021-06-24. Delta 6 days. 10.08%
Bought on 2021-07-07 & sold on 2021-07-12. Delta 5 days. 5.07%
Bought on 2021-07-16 & sold on 2021-07-23. Delta 7 days. 13.03%
Bought on 2021-08-18 & sold on 2021-08-23. Delta 5 days. 13.01%
Bought on 2021-10-12 & sold on 2021-10-14. Delta 2 days. 10.8%
Bought on 2021-11-26 & sold on 2021-12-07. Delta 11 days. 17.3%
Bought on 2021-12-20 & sold on 2021-12-23. Delta 3 days. 17.16%
Number of trades: 11
Percentage average when price opens higher than previous close: 14.616364

ticker: FAS
Bought on 2021-01-29 & sold on 2021-02-03. Delta 5 days. 12.86%
Bought on 2021-03-24 & sold on 2021-03-30. Delta 6 days. 9.51%
Bought on 2021-04-20 & sold on 2021-04-23. Delta 


ticker: MIDU
Bought on 2021-01-29 & sold on 2021-02-04. Delta 6 days. 14.87%
Bought on 2021-02-18 & sold on 2021-02-24. Delta 6 days. 9.62%
Bought on 2021-03-24 & sold on 2021-03-30. Delta 6 days. 11.48%
Bought on 2021-05-12 & sold on 2021-05-26. Delta 14 days. 9.4%
Bought on 2021-06-18 & sold on 2021-06-25. Delta 7 days. 13.71%
Bought on 2021-07-19 & sold on 2021-07-21. Delta 2 days. 12.42%
Bought on 2021-08-19 & sold on 2021-08-24. Delta 5 days. 9.26%
Bought on 2021-09-17 & sold on 2021-09-23. Delta 6 days. 2.76%
Bought on 2021-09-30 & sold on 2021-10-07. Delta 7 days. 7.33%
Bought on 2021-10-27 & sold on 2021-11-01. Delta 5 days. 11.45%
Bought on 2021-12-03 & sold on 2021-12-07. Delta 4 days. 11.28%
Bought on 2021-12-16 & sold on 2021-12-22. Delta 6 days. 4.63%
Number of trades: 12
Percentage average when price opens higher than previous close: 9.850833

ticker: MEXX
Bought on 2021-01-29 & sold on 2021-02-08. Delta 10 days. 14.09%
Bought on 2021-02-25 & sold on 2021-03-02. Delta 5 

In [138]:

(df.iloc[i]['date']-df.iloc[i]['date']).days



-5

In [ ]:
returnP1 = []; returnP2 = []
# setting the outputs in the df
for i in range(len(df)-1):
    if df.iloc[i]['close'] < df.iloc[i+1]['open']:
#         print("close date: %s" % (df.index[i]))
#         print("open date: %s" % (df.index[i+1]))
#         print()

        returnP1.append(df.iloc[i+1]['close_per1'])
    
        if df.iloc[i+1]['close_per1'] < 0:
            returnP2.append(-0.10)
        else:
            returnP2.append(df.iloc[i+1]['close_per1'])
#             returnP2.append(100*(df.iloc[i+1]['open']-df.iloc[i]['close'])/abs(df.iloc[i]['close']))
            
avgP1 = np.average(returnP1)
avgP2 = np.average(returnP2)

In [19]:
print("Number of trade days: %d" % (len(df)))
print("Number of trades: %d" % (len(returnP2)))
print("Percentage average when price opens higher than previous close: %f" % (avgP1))
print("Average return if loss limit is 0.1 percent: %f" % (avgP2))

Number of trade days: 239
Number of trades: 138
Percentage average when price opens higher than previous close: 1.668737
Average return if loss limit is 0.1 percent: 2.395377


In [12]:
print(100*(df.iloc[i]['close']-df.iloc[i]['open'])/abs(df.iloc[i]['open']))
print(df.iloc[i]['open'])
print(df.iloc[i]['close'])

-0.04696045478285022
85.18000030517578
85.13999938964844


In [1]:
import alpaca_trade_api as tradeapi
import requests
import time
import numpy as np
from datetime import datetime, timedelta
from pytz import timezone

In [23]:

BASE_URL = 'https://paper-api.alpaca.markets'
PUB_KEY = 'PK5L3FU9VKRXYAPYWJ6I'
SEC_KEY = 'vp0FJH8BZmBisyi2jjjwxMEjDHRAw1RdQfUMwS4M'
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL)



symb = "FAS"


market_data = api.get_barset(symb, 'day', limit=5) # Get one bar object for each of the past 5 minutes

close_list = [] # This array will store all the closing prices from the last 5 minutes
for bar in market_data[symb]:
    close_list.append(bar.c) # bar.c is the closing price of that bar's time interval

close_list = np.array(close_list, dtype=np.float64) # Convert to numpy array
ma = np.mean(close_list)
last_price = close_list[4] # Most recent closing price

# print("Moving Average: " + str(ma))
# print("Last Price: " + str(last_price))

print(market_data['FAS'][4].c)

In [146]:
import alpaca_trade_api as tradeapi
BASE_URL = 'https://paper-api.alpaca.markets'
PUB_KEY = 'PK92I7HHN7R3TNHJSJG8'
SEC_KEY = 'To6M3zJRMYfaBriBAtPru9E2uSg7MnwPLiy5CO4G'
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL)


In [151]:
api.get_quotes("AAPL", "2021-06-08", "2021-06-08", limit=3).df

,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,tape
timestamp,,,,,,,,
2021-06-08 08:00:00.082800+00:00,P,129.8,1,P,123.8,1,[?],C
2021-06-08 08:00:00.082800+00:00,P,143.0,1,P,120.0,1,[?],C
2021-06-08 08:00:00.082800+00:00,P,129.7,1,P,124.2,1,[?],C


In [149]:


# Get daily price data for AAPL over the last 5 trading days.
l = 1000
barset = api.get_barset('AAPL', 'minute', limit=l)
aapl_bars = barset['AAPL']

# See how much AAPL moved in that timeframe.
week_open = aapl_bars[0].o
week_close = aapl_bars[-1].c
percent_change = (week_close - week_open) / week_open * 100
print('AAPL moved {}% over the last {} minutes'.format(percent_change,l))

AAPL moved 1.3768740786069897% over the last 1000 minutes


In [150]:
aapl_bars.df

,open,high,low,close,volume
time,,,,,
2021-12-30 11:55:00-05:00,179.755,179.755,179.755,179.755,131
2021-12-30 11:56:00-05:00,179.790,179.790,179.790,179.790,300
2021-12-30 11:57:00-05:00,179.770,179.770,179.750,179.750,512
2021-12-30 11:58:00-05:00,179.750,179.750,179.740,179.750,1407
2021-12-30 11:59:00-05:00,179.750,179.820,179.750,179.815,1621
...,...,...,...,...,...
2022-01-03 16:08:00-05:00,182.150,182.180,182.150,182.180,200
2022-01-03 16:09:00-05:00,182.170,182.170,182.160,182.160,200
2022-01-03 16:14:00-05:00,182.190,182.190,182.190,182.190,100
